In [2]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
pip install accelerate transformers

# Requirement
I want a small model that responds with a SQL given the context and the user query. I want model to avoid adding any additional information other than SQL itself.

In [4]:
# Packages
import torch
from transformers import pipeline

# Define the model to infer
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")


# Conversational message for the chat model
messages = [
    {"role": "system","content": "CREATE TABLE head(age INTEGER)"},
    {"role": "user", "content": "How many heads of the departments are older than 56?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


<|system|>
CREATE TABLE head(age INTEGER)</s>
<|user|>
How many heads of the departments are older than 56?</s>
<|assistant|>
I don't have access to the latest data or the current head count of the departments. However, in general, the heads of departments usually hold positions for a long time, and as such, it's common for them to have a long tenure. Therefore, it is not uncommon for the heads of departments to be older than 56 years old. The specific head count of the departments can be obtained by contacting the respective organizations.


# Prompt Engineering
Let's do some prompt engineering here to help model generate just the SQL we want

In [7]:
# Prompt engineering
Additional_system_prompt = "You can only reply in SQL query language hence provide only SQL for user's query given this context -->"

messages = [
    {"role": "system","content": f"{Additional_system_prompt} CREATE TABLE head(age INTEGER)"},
    {"role": "user", "content": "How many heads of the departments are older than 56?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You can only reply in SQL query language hence provide only SQL for user's query given this context --> CREATE TABLE head(age INTEGER)</s>
<|user|>
How many heads of the departments are older than 56?</s>
<|assistant|>
The given context doesn't provide a specific number for the heads of the departments. It only states that the age of one head is greater than 56. You can use SQL query language to retrieve the total number of heads of the departments that are older than 56 by running the query like this:

```
SELECT COUNT(*)
FROM head
WHERE age > 56
```

This query will return the number of heads of the departments that are older than 56.


More Prompt engineering

In [8]:
# More prompt engineering
Additional_system_prompt = """You can only reply in SQL query language hence provide only SQL for user's query given the context.
  e.g.  Context : CREATE TABLE schools(name STRING, students INTEGER)
        Question : How many schools have more than 100 students?
        Answer : SELECT COUNT(*) FROM schools WHERE students > 100
-->"""

messages = [
    {"role": "system","content": f"{Additional_system_prompt} CREATE TABLE head(age INTEGER)"},
    {"role": "user", "content": "How many heads of the departments are older than 56?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You can only reply in SQL query language hence provide only SQL for user's query given the context. 
  e.g.  Context : CREATE TABLE schools(name STRING, students INTEGER)
        Question : How many schools have more than 100 students?
        Answer : SELECT COUNT(*) FROM schools WHERE students > 100
--> CREATE TABLE head(age INTEGER)</s>
<|user|>
How many heads of the departments are older than 56?</s>
<|assistant|>
To calculate the number of heads of the departments older than 56, you can use the following SQL query:

```
SELECT COUNT(*) FROM departments WHERE age > 56;
```

In the above query, "departments" is the name of the table and "age" is the column name. The "COUNT(*) FROM" clause counts the number of rows in the table and the "WHERE" clause filters the rows based on the specified condition. The `COUNT(*)` function returns the number of rows in the table, and the `COUNT(*) FROM` clause counts the number of rows returned by the `COUNT(*)` function.
